In [34]:

#Objectifs:
#  Programmer sous R:
#    1) Une fonction pour la (TRAIN-TEST) validation répétée
#    2) Une fonction pour la validation croisée (K feuilles)
#    3) Une fonction pour la validation croisée imbriquée
#    4) Une fonction pour le Leave One Out (avec k = n)
# 
# deux critères à valider: moyenne des performances et écart-type des 

# construction de modèles (ou librairies de notre choix):
# library(rpart) = algos de décisions -> CART: classification and regression tree (classif ou regression)
# library(MASS) = algo de décisions -> LDA : linear decision analysis: (classif binaire)
# library(FNN) : K plus proche voisin (k-PPV)
# tree <- rpart(X10 ~ ., data= mesData, method= 'class')  ici ~ = en fonction de et . = toutes les autres variables
# tree <- rpart(X10 ~ ., data= mesData, method= 'class', control=rpart.control(minsplit=50), cp= val entre 0 et 1)  ici ~ = en fonction de et . = toutes les autres variables
# predict(tree, mesNouvellesDonnées, type='class')
# predict(tree, mesNouvellesDonnées, type='prob')
# JEUX de données madoc: Breiman: wave5000

# attente livrable: les fonctions, mini rapport explication fonction et comparaison des modèles(1,2,3,4) :robustesse sur les données de Breiman

# en sortie un vecteur mélanger de 1 à n: en entré un entier
# fonction sample déjà défini


# Question 1
#data = dataframe avec une colonne réel
#k = le nombre de répétition
#predictColumn = string = nom de la colonne X10
library(rpart)
validationRepeteeRPART = function (data, k) {
  errorRate = c();
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData$X10~.,modalData, method="class")
    prediction = predict(tree, testData, type="class")
    
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction)
    
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

library(MASS)

validationRepeteeLDA = function (data, k) {
  errorRate = c()
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = lda(modalData$X10~.,modalData, method="moment")
    prediction = predict(tree, testData, type="moment")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction$class)
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

#Exercice 2
#data = dataframe
#partitions = integer
validationCroiseeRPART = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData_step2$X10~.,modalData_step2, method="class")
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}




validationCroiseeLDA = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = lda(X10~.,modalData_step2)
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction$class)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}


#appel de fonction validationCroisee défini précédemment
#en paramètre k = le nombre de répétition
validationCroiseeRepetee = function (data, partitions, k) {
  robustesseMean = 0
  errorRateMean = 0
  
  for (i in 1:k) {
    robustesseMean = robustesseMean + validationCroiseeRPART(data, partitions, 1)
    errorRateMean = errorRateMean + validationCroiseeRPART(data, partitions, 0)
  }
  
  robustesseMean = robustesseMean/k
  errorRateMean = errorRateMean/k
  
  return(c(robustesseMean, errorRateMean))
  
}

#data = dataframe
#partitions = nombre de partition où on va split le dataset
validationCroiseeImpliquee = function(data, partitions) {
  
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  listErrorRate = c()
  
  for (i in 1:partitions) {
    
    errorRate = c()
    
    for (i in 1:partitions-1) {
 
      tempSetPartitions = setPartitions
      currentTestData = setPartitions[[i]]
      currentModalDataList = tempSetPartitions[-i]
      #fusion dataframe
      currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
      
      for (j in 3:(partitions-2)) {
        currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
      }
      
      numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
      numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
      
      testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
      modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
      
      #on calcule la précision (le taux d'erreurs)
      tree = lda(X10~.,modalData_step2)
      prediction = predict(tree, testData_step2, type="class")
      #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
      contingencyTable = table(testData_step2$X10, prediction$class)
      
      total = 0
      sumError = 0
      for (i in 1:nrow(contingencyTable)) {
        for (j in 1:ncol(contingencyTable)) {
          if (i != j) {
            sumError = sumError + contingencyTable[i,j]
          }
          total = total + contingencyTable[i,j]
        }
      }
      errorRate = c(errorRate, c(sumError/total))
    }
    
    listErrorRate = c(c(listErrorRate), mean(errorRate))
    
  }
  
  return(listErrorRate)
  
}

#appel de la fonction validationCroisee
#en paramètre data: un dataframe
leaveOneOut = function(data) {
  numberOfrows = nrow(data)
  return(c(validationCroiseeRPART(data, numberOfrows, 0), validationCroiseeRPART(data, numberOfrows, 1)))
}


#appel de tous les fonctions définies précédemment

algorithmSelection = function(data, K) {
  
  validationCroiseeRPARTErrorRate = validationRepeteeLDA(data, K)
  validationCroiseeLDAErrorRate = validationRepeteeRPART(data, K)
  
  if (validationCroiseeRPARTErrorRate > validationCroiseeLDAErrorRate) {
    return("SOLUTION: LDA")
  } else {
    return("SOLUTION: RPART")
  }
  
  return("ERROR")
}

validationCroiseeGetModel = function(data, partitions, colonneCible, errorRateThreshold = 0.3) {
  errorRate = c()
  #on m\u{fffd}lange les donn\u{fffd}es
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la pr\u{fffd}cision (le taux d'erreurs)
    #formulaString = paste("modalData_step2$", colonneCible, "~.",sep="")
    #formulaCible= as.formula(formulaString)
    #tree = rpart(formulaCible, modalData_step2, method="class", control=rpart.control(minsplit=5,cp=0))
      
    tree = rpart(modalData_step2$X1~., modalData_step2, method="class", control=rpart.control(minsplit=5,cp=0))
    prediction = predict(tree, testData_step2, type="class")
      
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    #formulaString = paste("testData_step2", colonneCible,sep="$")
    #formulaCible2 = as.formula(formulaString)
      
    contingencyTable = table(testData_step2$X1, prediction)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la m\u{fffd}me que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (mean(errorRate) <= errorRateThreshold) {
    listResult = list(mean(errorRate), tree)
    print("Error Rate < 0.3")
    print(mean(errorRate))
    return(listResult)
  }else{
    listResult = list(0, tree)
    print("Error Rate > 0.3, WRONG MODEL")
    print(mean(errorRate))
    return(listResult)
  }
}

#Variable cible : PREDICTION CHIFFRE D'AFFAIRES
GetModelChiffreAffaires = function(data, partitions){
  result = 0
  while(result == 0){
    print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,4,5,6,7,8,9)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1")
    result = listResult[[1]]
  }
  print(listResult)
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionChiffreAffaire = predict(listResult[[2]], donneesTest, type="class")
  print(predictionChiffreAffaire)
  return (predictionChiffreAffaire)
}

#Variable cible : CAPACITE D'EMPRUNT
GetModelCapaciteEmprunt = function(data, partitions){
  result = 0
  while(result == 0){
    print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,4,5,6,7,8,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.6)
    result = listResult[[1]]
  }
  print(listResult)
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionCapaciteEmprunt = predict(listResult[[2]], donneesTest, type="class")
  print(predictionCapaciteEmprunt)
  return (predictionCapaciteEmprunt)
}

GetModelSecteurOne = function(data, partitions){
#Variable cible : SECTEUR 1
  result = 0
  while(result == 0){
    print("generating new model ...")
    donneesAppr = data[ , -c(1,3,4,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    donneesAppr
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1")
    result = listResult[[1]]
  }
  print(listResult)
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  donneesTest
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionSecteurOne = predict(listResult[[2]], donneesTest, type="class")
  print(predictionSecteurOne)
  return (predictionSecteurOne)
}

GetModelSecteurTwo = function(data, partitions){
#Variable cible : SECTEUR 2
  result = 0
  while(result == 0){
    print("generating new model ...")
    donneesAppr = data[ , -c(1,2,4,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    donneesAppr
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1")
    result = listResult[[1]]
  }
  print(listResult)
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  donneesTest
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionSecteurTwo = predict(listResult[[2]], donneesTest, type="class")
  print(predictionSecteurTwo)
  return (predictionSecteurTwo)
}

GetModelSecteurParticulier = function(data, partitions){
#Variable cible : SECTEUR PARTICULIER
  result = 0
  while(result == 0){
    print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    donneesAppr
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1")
    result = listResult[[1]]
  }
  print(listResult)
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  donneesTest
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionSecteurParticulier = predict(listResult[[2]], donneesTest, type="class")
  print(predictionSecteurParticulier)
  return (predictionSecteurParticulier)
}

In [35]:
apprentissage <- read.table("apprentissage.csv", sep = "," ,header = FALSE)
test <- read.csv("test.csv", sep = "," ,header = FALSE)
# Remplacer les NA par des 0
apprentissage[is.na(apprentissage)]<-0
test[is.na(test)]<-0

predictionChiffreAffaire = GetModelChiffreAffaires(apprentissage, 10)

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.8333333
[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.65
[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate < 0.3"
[1] 0.2166667
[[1]]
[1] 0.2166667

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 16 0 (0.36 0.08 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04)  
   2) X2< 0.5 9  0 0 (1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0) *
   3) X2>=0.5 16 14 17500 (0 0.12 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062)  
     6) X2< 1.5 5  3 17500 (0 0.4 0.2 0.2 0.2 0 0 0 0 0 0 0 0 0 0 0)  
      12) X28< 1.02 2  0 17500 (0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0) *
      13) X28>=1.02 3  2 20000 (0 0 0.33 0.33 0.33 0 0 0 0 0 0 0 0 0 0 0) *
     7) X2>=1.5 11 10 35000 (0 0 0 0 0 0.091 0.091 0.091 0.091 0.091 0.091 0.091 0.091 0.091 0.091 0.091)  
      14) X2< 5 4  3 35000 (0 0 0 0 0 0.25 0.25 0.25 0.25 0 0 0 0 0 0 0) *
      15) X2>=5 7  6 105000 (0 0 0 0 0 0 0 0 0 0.14 0.14 0.14 0.14 0.14 0.14 0.14)  
        30) X2< 8.5 3  2 105000 (0 0 0 0 0 0 0 0 0 0.33 0.33 0.33 0 0 0 0) *
        31) X2>=8.5 4 

In [36]:
predictionCapaciteEmprunt = GetModelCapaciteEmprunt(apprentissage, 10)

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9666667
[1] "generating new model ..."


...

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate < 0.3"
[1] 0.55
[[1]]
[1] 0.55

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 13 10000 (0.04 0.24 0.48 0.08 0.08 0.04 0.04)  
   2) X30< 5.1 11  5 5000 (0 0.55 0.27 0 0.091 0 0.091)  
     4) X2>=0.5 6  1 5000 (0 0.83 0 0 0.17 0 0) *
     5) X2< 0.5 5  2 10000 (0 0.2 0.6 0 0 0 0.2)  
      10) X6< 32.725 3  0 10000 (0 0 1 0 0 0 0) *
      11) X6>=32.725 2  1 5000 (0 0.5 0 0 0 0 0.5) *
   3) X30>=5.1 14  5 10000 (0.071 0 0.64 0.14 0.071 0.071 0)  
     6) X15< 16.95 10  1 10000 (0.1 0 0.9 0 0 0 0) *
     7) X15>=16.95 4  2 15000 (0 0 0 0.5 0.25 0.25 0) *

    1     2     3     4     5     6     7     8     9    10    11    12    13 
10000 15000 15000  5000 10000 15000  5000 15000  5000 15000 10000 10000 15000 
   14    15    16    17    18    19    20    21    22    23    24    25    26 
 5000 10000 10000 15000  5000 15000 10000  5000  5000  5000 10000 10000 10000 
   27    28    29    30    31    32    33    34    35    36

In [45]:
predictionSecteurOne = GetModelSecteurOne(apprentissage, 10)

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate < 0.3"
[1] 0.3
[[1]]
[1] 0.3

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 25 2 0 (0.9200000 0.0800000)  
  2) X15< 3929.665 23 0 0 (1.0000000 0.0000000) *
  3) X15>=3929.665 2 0 1 (0.0000000 1.0000000) *

   1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16 
   0    0    0    0    0    0    0    0    0    0    0    0    0    1    0    0 
  17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32 
   0    0    1    0    1    0    1    0    0    0    0    0    0    0    1    0 
  33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48 
   0    0    0    0    0    0    0    0    1    0    0    0    0    0    0    0 
  49   50   51   52   53   54   55   56   57   58   59   60   61   62   63   64 
   0    0    0    0    0    1    0    0    0    0    1    0    0    0    0    0 
  65   66   67   68   69   70   71   72   73   74   75   76   77   78   79   80 
   0

In [43]:
predictionSecteurTwo = GetModelSecteurTwo(apprentissage, 10)

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate < 0.3"
[1] 0.1833333
[[1]]
[1] 0.1833333

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 5 0 (0.8000000 0.2000000)  
   2) X14< 7818.15 23 3 0 (0.8695652 0.1304348)  
     4) X9>=4.225 13 0 0 (1.0000000 0.0000000) *
     5) X9< 4.225 10 3 0 (0.7000000 0.3000000)  
      10) X23< 67.605 8 1 0 (0.8750000 0.1250000) *
      11) X23>=67.605 2 0 1 (0.0000000 1.0000000) *
   3) X14>=7818.15 2 0 1 (0.0000000 1.0000000) *

   1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16 
   0    0    0    0    0    0    0    0    0    0    1    0    1    0    0    1 
  17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32 
   0    0    0    0    0    0    0    0    0    0    1    0    0    0    0    0 
  33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48 
   0    0    0    1    0    0    0    0    1    0    1    0    0    0    1    0 
  49   50   51   52   53   54   

In [58]:
predictionSecteurParticulier = GetModelSecteurParticulier(apprentissage, 10)
apprentissage
test

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

ERROR: Error in cbind(yval2, yprob, nodeprob): le nombre de lignes des matrices doit correspondre (voir argument 2)
